In [ ]:
!pip install BisPy
!pip install ogb
!pip install torch-geometric

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import pandas as pd
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB, Actor
import os
import bispy
from bispy import compute_maximum_bisimulation, Algorithms
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti_partition
from bispy.dovier_piazza_policriti.ranked_partition import RankedPartition
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti, collapse, build_block_counterimage, split_upper_ranks
from bispy.utilities.rank_computation import compute_rank
from bispy.utilities.graph_decorator import decorate_nx_graph, decorate_bispy_graph,as_bispy_graph,to_tuple_list
from bispy.utilities import *
import numpy as np
from sklearn.cluster import KMeans

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Create an empty NetworkX graph
from torch_geometric.utils import from_networkx
G = nx.DiGraph()
node_map = {}

def get_key(val, node_map):
    for key, value in node_map.items():
        if val == value:
            return key
    return "key doesn't exist"


# Driver Code
# my_dict = {"Java": 100, "Python": 112, "C": 11}

# print(get_key(100))
# print(get_key(11))

# Read node features and labels from 'cora.content'
with open('gdrive/My Drive/GNN_Sparsification/Algorithms/cora.content', 'r') as content_file:
    i = 0
    for line in content_file:
        data = line.strip().split('\t')
        node_id = data[0]
        features = [float(x) for x in data[1:-1]]
        label = data[-1]
        node_map[i] = node_id
        # print(node_id)
        # Add node to the graph with features and label
        G.add_node(i, features=features, label=label)
        i+=1

# Read edges from 'cora.cites'
with open('gdrive/My Drive/GNN_Sparsification/Algorithms/cora.cites', 'r') as cites_file:
    for line in cites_file:
        source, target = line.strip().split('\t')
        source = get_key(source, node_map)
        target = get_key(target, node_map)
        G.add_edge(source, target)


data_path = 'gdrive/My Drive/GNN_Sparsification/Algorithms/cora.content'  # Update with the correct path
# column_names = ["paper_id"] + [f"attr_{i}" for i in range(1433)] + ["subject"]
column_names = [f"attr_{i}" for i in range(1433)] + ["subject"]
df = pd.read_csv(data_path, sep='\t', header=None, names=column_names)

# Display the first few rows
df = df.iloc[:,1:-1]

# Access a specific attribute for a specific node
# attribute_values_for_node_0 = df.loc[0, "attr_0":"attr_1432"]

# Access the subject label for a specific node
# subject_label_for_node_0 = df.loc[0, "subject"]
# Print basic graph information
data = from_networkx(G)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 2708
Number of edges: 5429


In [ ]:
# Cora
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB

dataset = Planetoid(root='/tmp/Cora', name='Cora')
# dataset = Yelp(root='Yelp')
cora = dataset[0]
cora.y = torch.squeeze(cora.y)
cora

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
cora.edge_index = data.edge_index
cora

Data(x=[2708, 1433], edge_index=[2, 5429], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
# Load tensor from .pt file
loaded_tensor = torch.load('ARXIV_SPAR.pt')
data.edge_index = loaded_tensor.edge_index

In [ ]:
data

Data(edge_index=[2, 5429], features=[2708, 1433], label=[2708], num_nodes=2708)

In [ ]:
import copy
from torch_geometric.utils import subgraph
train_mask = torch.rand(data.num_nodes) < 0.8
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
file_path = "gdrive/My Drive/GNN_Sparsification/20%_cora_train_mask.pt"

# Save the binary tensor mask to a .pt file
torch.save(train_mask, file_path)

In [ ]:
train_mask = torch.load('gdrive/My Drive/GNN_Sparsification/20%_cora_train_mask.pt')
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx


# Convert PyG graph to NetworkX graph
def pyg_to_networkx(pyg_data):
    edge_index = pyg_data.edge_index
    num_nodes = pyg_data.num_nodes
    edge_list = edge_index.t().tolist()
    G = nx.DiGraph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_list)
    return G

networkx_graph = pyg_to_networkx(cora)

# Radius-based K-hops Induced Subgraph

In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
# G = nx.Graph()
G = nx.DiGraph()

for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.neighbors(node))
    neighbors.update(networkx_graph.predecessors(node))

    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in nodes_to_find_neighbors:
          G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in nodes_to_find_neighbors:
          G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(neighbor, node)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0


# Diameter-based K-hops Induced Subgraph

In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
# G = nx.Graph()
G = nx.DiGraph()

for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.predecessors(node))

    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in nodes_to_find_neighbors:
          G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.predecessors(node):
        # if neighbor not in up_to_k_hop_neighbors_list:
        if neighbor not in nodes_to_find_neighbors:
          G.add_edge(neighbor, node)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0


In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 1546
Number of edges: 1665


82

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

import time

# Measure the execution time
start_time = time.time()

partition = compute_maximum_bisimulation(G)
len(partition)

end_time = time.time()
execution_time = end_time - start_time



print(f"Execution time: {execution_time} seconds")

Number of nodes: 2708
Number of edges: 5429
Execution time: 0.069183349609375 seconds


In [ ]:
cora

Data(x=[2708, 1433], edge_index=[2, 5429], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
cora_FGC = torch.load('gdrive/My Drive/GNN_Sparsification/Coarsened_CORA.pt')
cora_FGC

Data(x=[1372, 1433], edge_index=[1372, 1372], y=[1372])

In [ ]:
edge_indices

tensor([[   0,    0,    1,  ..., 1371, 1371, 1371],
        [   1,  815,  695,  ..., 1126, 1313, 1353]])

In [ ]:
cora_FGC.edge_index = edge_indices
cora_FGC

Data(x=[1372, 1433], edge_index=[2, 9527], y=[1372])

In [ ]:
torch.save(cora_FGC,'gdrive/My Drive/GNN_Sparsification/cora_FGC.pt')

In [ ]:
tuple_list = partition
def find_tuple_index_with_integer(list_of_tuples, integer):
    return next((index for index, tup in enumerate(list_of_tuples) if integer in tup), -1)

# Sort the list by the number of elements in each tuple
sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

indices = []
list_of_lists = [list(t) for t in sorted_list]

for ele in tested_elements:
  id = find_tuple_index_with_integer(sorted_list, ele)
  if len(list_of_lists[id]) > 1:
    list_of_lists[id].remove(ele)
    list_of_lists.append([ele])
    indices.append(len(list_of_lists)-1)
  else:
    indices.append(id)

list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]

In [ ]:
i = 0
while len(list_of_lists[i])>1:
    # print(i)

    Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
    threshold = 0.6 * np.max(Z[:, 2])
    clusters = fcluster(Z, threshold, criterion='distance')

    # Get cluster labels for each data point
    labels = clusters

    # Sample tuple to split
    data_tuple = list_of_tuples[i]

    # Create a dictionary to store split data
    split_data = {}

    # Iterate through the labels and tuple elements
    for label, value in zip(labels, data_tuple):
      if label not in split_data:
        split_data[label] = []
      split_data[label].append(value)

    # Convert the dictionary to a list of tuples (if needed)
    split_tuples = [tuple(values) for key, values in split_data.items()]
    list_of_tuples.pop(i)
    j = 0
    for values in split_tuples:
      if j == 0:
        list_of_tuples.insert(0, values)
      else:
        list_of_tuples.insert(len(list_of_tuples), values)
      j+=1

    i+=1

result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

# Alpha = 0.1


In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 74520
Number of edges: 185135


# Alpha = 0.2

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 54414
Number of edges: 177409


# Alpha = 0.3

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 43738
Number of edges: 161673


# 0.4

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 39537
Number of edges: 143608


# 0.5

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 37687
Number of edges: 134284


# 1

In [ ]:
result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 35114
Number of edges: 122843


In [ ]:
from collections import Counter
import random

new_x = torch.zeros(num_nodes,data.x.shape[1])
new_y = torch.zeros(num_nodes)
for i in range(len(list_of_tuples)):
  new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
  EC = list_of_tuples[i]
  L = []
  for ele in EC:
    L.append(int(data.y[ele].numpy()))

  keys = list(Counter(L).keys())
  frequencies = list(Counter(L).values())

  # Randomly select a key based on frequencies
  random_key = random.choices(keys, frequencies)[0]
  new_y[i] = int(random_key)

edges = list(BCG.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
data

Data(edge_index=[2, 5429], features=[2708, 1433], label=[2708], num_nodes=2708, x=[2708, 1433], y=[2708])

In [ ]:
data.y = cora.y

In [ ]:
from collections import Counter
import random
import copy
from collections import Counter

new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
new_y = torch.zeros(len(list(G.nodes)))
list_itr = list(G.nodes)
for i in range(len(list(G.nodes))):
  new_x[i,:] = data.x[list_itr[i],:]
  new_y[i] = int(data.y[list_itr[i]])

edges = list(G.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(G.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
torch.save(data_new1, 'gdrive/My Drive/GNN_Sparsification/20%_Induced_cora.pt')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(partition, tested_elements):
    tuple_dict = {}
    indices = []

    tuple_list = partition

    # Sort the list by the number of elements in each tuple
    sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

    list_of_lists = [list(t) for t in sorted_list]

    # Convert tuples to lists and create a dictionary with elements as keys and their corresponding tuple index as values
    for idx, tpl in enumerate(list_of_lists):
        tpl_list = list(tpl)
        for ele in tpl_list:
            tuple_dict[ele] = idx
        list_of_lists[idx] = tpl_list

    # Update the dictionary and indices based on tested elements
    for ele in tested_elements:
        if ele in tuple_dict:
            idx = tuple_dict[ele]
            if len(list_of_lists[idx]) > 1:
                list_of_lists[idx].remove(ele)
                list_of_lists.append([ele])
                tuple_dict[ele] = len(list_of_lists) - 1
                indices.append(len(list_of_lists) - 1)
            else:
                indices.append(idx)

    return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:].detach().numpy(), method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

# def alpha_Sparsification(list_of_lists, data, alpha):
#     list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
#     i = 0
#     while i<len(list_of_lists):
#         if len(list_of_lists[i])>1:
#             Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
#             threshold = alpha * np.max(Z[:, 2])
#             clusters = fcluster(Z, threshold, criterion='distance')
#             # Get cluster labels for each data point
#             labels = clusters
#             # Sample tuple to split
#             data_tuple = list_of_tuples[i]
#             # Create a dictionary to store split data
#             split_data = {}
#           # Iterate through the labels and tuple elements
#             for label, value in zip(labels, data_tuple):
#                 if label not in split_data:
#                     split_data[label] = []
#                 split_data[label].append(value)

#           # Convert the dictionary to a list of tuples (if needed)
#             split_tuples = [tuple(values) for key, values in split_data.items()]
#             list_of_tuples.pop(i)
#             j = 0
#             for values in split_tuples:
#                 if j == 0:
#                     list_of_tuples.insert(0, values)
#                 else:
#                     list_of_tuples.insert(len(list_of_tuples), values)
#                 j+=1

#         i+=1

#     result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
#     return list_of_tuples, result_dict

In [ ]:
def Sparisified_G_NX(list_of_tuples, result_dict, data):
  BCG = nx.DiGraph()

  for i in range(len(list_of_tuples)):
    BCG.add_node(i)

  for edge in G.edges():
      source, target = edge
      # if cosine_similarity(data.x[source].numpy().reshape(1, -1), data.x[target].numpy().reshape(1, -1)) > 0.5:
      BCG.add_edge(result_dict[source], result_dict[target])

  num_nodes = BCG.number_of_nodes()
  num_edges = BCG.number_of_edges()



  print("Number of nodes:", num_nodes)
  print("Number of edges:", num_edges)


  return BCG

In [ ]:
from torch_geometric.data import Data

def Sparisified_G_Alpha(BCG, data, list_of_tuples):
  num_nodes = BCG.number_of_nodes()
  new_x = torch.zeros(num_nodes,data.x.shape[1])
  new_y = torch.zeros(num_nodes)
  for i in range(len(list_of_tuples)):
    new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    # new_x[i,:] = torch.sum(data.x[list(list_of_tuples[i]),:], axis = 0)
    # new_x[i, :] = torch.tensor(np.median(data.x[list(list_of_tuples[i]), :], axis=0))
    # new_x[i, :] = torch.max(data.x[list(list_of_tuples[i]), :], axis=0).values
    # new_x[i, :] = torch.min(data.x[list(list_of_tuples[i]), :], axis=0).values


    EC = list_of_tuples[i]
    L = []
    for ele in EC:
      L.append(int(data.y[ele].numpy()))

    keys = list(Counter(L).keys())
    frequencies = list(Counter(L).values())

    # Randomly select a key based on frequencies
    random_key = random.choices(keys, frequencies)[0]
    new_y[i] = int(random_key)

  print("Completed_Flag")
  edges = list(BCG.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

  data_new1 = Data()
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  return data_new1

In [ ]:
def Sparisified_G_Alpha_0(G, data):

  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))
  list_itr = list(G.nodes)
  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[list_itr[i],:]
    new_y[i] = int(data.y[list_itr[i]])

  edges = list(G.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(G.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


  data_new1 = Data()
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  return data_new1

In [ ]:
import copy
from collections import Counter
alpha_list = [0, 0.1, 1]

for alpha in alpha_list:
  # print(alpha)
  if alpha == 0:
    start_time = time.time()
    data_new_1 = Sparisified_G_Alpha_0(G, cora)
    end_time = time.time()
    execution_time_SPGS = end_time - start_time
    print(f"Execution time: {execution_time_SPGS} seconds")

    tested_elements_set = set(tested_elements)
    indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]


  if alpha > 0 and alpha < 1:
    print(alpha)
    if alpha == alpha_list[1]:
      start_time_1 = time.time()
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)


    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, cora, alpha)
    # end_time_1 = time.time()
    # execution_time_1 = end_time_1 - start_time_1
    # print(f"Execution time of alpha_Sparsification: {execution_time_1} seconds")

    BCG = Sparisified_G_NX(list_of_tuples, result_dict, cora)
    data_new_1 = Sparisified_G_Alpha(BCG, cora, list_of_tuples)
    end_time_2 = time.time()
    execution_time_2 = end_time_2 - start_time_1
    print(f"Execution time of  (alpha, r) SPGC: {execution_time_2} seconds")

  if alpha == 1:
    l_o_t = [tuple(inner_list) for inner_list in list_of_lists]
    r_d = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
    start_time_3 = time.time()
    BCG = Sparisified_G_NX(l_o_t, r_d, cora)
    data_new_1 = Sparisified_G_Alpha(BCG, cora, l_o_t)
    end_time_3 = time.time()
    execution_time_3 = end_time_3 - start_time_3
    print(f"Execution time of SPGC: {execution_time_3} seconds")

Execution time: 0.08926892280578613 seconds
0.1
Number of nodes: 2686
Number of edges: 5402
Completed_Flag
Execution time of  (alpha, r) SPGC: 0.9183559417724609 seconds
Number of nodes: 2686
Number of edges: 5402
Completed_Flag
Execution time of SPGC: 0.545405387878418 seconds


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
# import torch

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super().__init__()

#     self.conv1 = GCNConv(dataset.num_node_features, 128,aggr = 'mean')
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GCNConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GCNConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GCN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# print("Graph Convolutional Network (GCN):")
# GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 40)
  (bc3): BatchNorm(40)
)

# 4-layer GCN

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GCNConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GCNConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = GCNConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 128)
  (bc3): BatchNorm(128)
  (conv4): GCNConv(128, 40)
  (bc4): BatchNorm(40)
)

# 3 Layer GCN

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GCNConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GCNConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 40)
  (bc3): BatchNorm(40)
)

# 2-layer GCN

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GCNConv(128, 40)
    self.bc2 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 40)
  (bc2): BatchNorm(40)
)

# 4-layer GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GATConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GATConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = GATConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 128, heads=1)
  (bc3): BatchNorm(128)
  (conv4): GATConv(128, 40, heads=1)
  (bc4): BatchNorm(40)
)

# 3-layer GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GATConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GATConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 40, heads=1)
  (bc3): BatchNorm(40)
)

# 2-layer GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GATConv(128, 40)
    self.bc2 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 40, heads=1)
  (bc2): BatchNorm(40)
)

# 4-layer SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch
from torch.nn import Module, Dropout

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = SAGEConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = SAGEConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = SAGEConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(128, 128, aggr=mean)
  (bc1): BatchNorm(128)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (bc2): BatchNorm(128)
  (conv3): SAGEConv(128, 128, aggr=mean)
  (bc3): BatchNorm(128)
  (conv4): SAGEConv(128, 40, aggr=mean)
  (bc4): BatchNorm(40)
)

# 3-layer SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch
from torch.nn import Module, Dropout

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = SAGEConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = SAGEConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(128, 128, aggr=mean)
  (bc1): BatchNorm(128)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (bc2): BatchNorm(128)
  (conv3): SAGEConv(128, 40, aggr=mean)
  (bc3): BatchNorm(40)
)

# 2-layer SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch
from torch.nn import Module, Dropout

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = SAGEConv(128, 40)
    self.bc2 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(128, 128, aggr=mean)
  (bc1): BatchNorm(128)
  (conv2): SAGEConv(128, 40, aggr=mean)
  (bc2): BatchNorm(40)
)

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, SAGEConv
# import torch

# class GraphSAGE(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = SAGEConv(dataset.num_node_features, 128)
#     # self.conv2 = GCNConv(40, 20)
#     self.conv2 = SAGEConv(128, dataset.num_classes)
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matri
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)

#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GraphSAGE()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("GraphSAGE:")
# GraphSAGE()

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, SAGEConv
# import torch

# class GAT(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = GATConv(dataset.num_node_features, 128,aggr = 'mean')
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GATConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GATConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)

#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GAT()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("GAT:")
# GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 40, heads=1)
  (bc3): BatchNorm(40)
)

In [ ]:
# from torch_geometric.nn import GraphSAGE, GATConv
# import torch

# class GAT(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = GATConv(dataset.num_node_features, 80, aggr = 'mean')
#     # self.conv2 = GCNConv(40, 20)
#     self.conv2 = GATConv(80, dataset.num_classes, aggr = 'mean')
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     # x = torch.nn.functional.dropout(x, training=self.training)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     # x = self.conv3(x, edge_index)
#     # x = torch.nn.functional.relu(x)
#     # # x = self.conv4(x, edge_index)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GAT()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("Graph Attention Networks(GAT):")
# GAT()

Graph Attention Networks(GAT):


GAT(
  (conv1): GATConv(128, 80, heads=1)
  (conv2): GATConv(80, 40, heads=1)
)

# Pre-trained GCN

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out, data.y)
  correct = compute_accuracy(out.argmax(dim=1), data.y)
  acc = int(correct) / len(data.y)
  losses.append(loss.item())
  accuracies.append(acc)

  # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

# torch.save(model.state_dict(), model_save_path_GCN)

Epoch: 10, Loss: 2.3812, Training Acc: 0.5022


KeyboardInterrupt: 

In [ ]:
out

tensor([[-11.7404, -11.7409, -11.8637,  ..., -12.0007, -11.8945, -11.9308],
        [ -3.6977,  -4.0338,  -2.7796,  ...,  -4.2936,  -4.1874,  -3.2253],
        [ -3.7156,  -3.8400,  -3.9628,  ...,  -4.0999,  -3.9936,  -3.3743],
        ...,
        [ -4.3302,  -4.3408,  -2.4261,  ...,  -4.6980,  -4.5918,  -4.6281],
        [ -3.9483,  -3.9488,  -4.0716,  ...,  -4.2086,  -4.1024,  -4.1386],
        [ -4.5502,  -4.0852,  -4.6735,  ...,  -2.4233,  -4.0455,  -4.7405]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
  correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
  acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
  losses.append(loss.item())
  accuracies.append(acc)

  # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

# torch.save(model.state_dict(), model_save_path_GCN)

NameError: name 'test_mask' is not defined

# Pre-trained GAT

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
  correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
  acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
  losses.append(loss.item())
  accuracies.append(acc)

  # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

torch.save(model.state_dict(), model_save_path_GAT)

Epoch: 10, Loss: 2.1640, Training Acc: 0.5231
Epoch: 20, Loss: 1.8687, Training Acc: 0.5670
Epoch: 30, Loss: 1.6964, Training Acc: 0.5911
Epoch: 40, Loss: 1.5726, Training Acc: 0.6065
Epoch: 50, Loss: 1.4784, Training Acc: 0.6180
Epoch: 60, Loss: 1.4030, Training Acc: 0.6285
Epoch: 70, Loss: 1.3402, Training Acc: 0.6388
Epoch: 80, Loss: 1.2863, Training Acc: 0.6480
Epoch: 90, Loss: 1.2381, Training Acc: 0.6573
Epoch: 100, Loss: 1.1968, Training Acc: 0.6646
Epoch: 110, Loss: 1.1584, Training Acc: 0.6737
Epoch: 120, Loss: 1.1229, Training Acc: 0.6809
Epoch: 130, Loss: 1.0938, Training Acc: 0.6877
Epoch: 140, Loss: 1.0634, Training Acc: 0.6961
Epoch: 150, Loss: 1.0386, Training Acc: 0.7017
Epoch: 160, Loss: 1.0143, Training Acc: 0.7087
Epoch: 170, Loss: 0.9943, Training Acc: 0.7121
Epoch: 180, Loss: 0.9774, Training Acc: 0.7183
Epoch: 190, Loss: 0.9569, Training Acc: 0.7230
Epoch: 200, Loss: 0.9538, Training Acc: 0.7211
Epoch: 210, Loss: 0.9290, Training Acc: 0.7298
Epoch: 220, Loss: 0.91

# Pre-trained GraphSage

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
  correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
  acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
  losses.append(loss.item())
  accuracies.append(acc)

  # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

torch.save(model.state_dict(), model_save_path_GraphSage)

Epoch: 10, Loss: 2.1108, Training Acc: 0.5376
Epoch: 20, Loss: 1.8243, Training Acc: 0.5744
Epoch: 30, Loss: 1.6388, Training Acc: 0.5985
Epoch: 40, Loss: 1.5055, Training Acc: 0.6155
Epoch: 50, Loss: 1.4052, Training Acc: 0.6304
Epoch: 60, Loss: 1.3247, Training Acc: 0.6428
Epoch: 70, Loss: 1.2566, Training Acc: 0.6551
Epoch: 80, Loss: 1.2061, Training Acc: 0.6641
Epoch: 90, Loss: 1.1482, Training Acc: 0.6765
Epoch: 100, Loss: 1.1044, Training Acc: 0.6855
Epoch: 110, Loss: 1.0570, Training Acc: 0.6959
Epoch: 120, Loss: 1.0236, Training Acc: 0.7036
Epoch: 130, Loss: 0.9921, Training Acc: 0.7107
Epoch: 140, Loss: 0.9477, Training Acc: 0.7231
Epoch: 150, Loss: 0.9181, Training Acc: 0.7308
Epoch: 160, Loss: 0.8918, Training Acc: 0.7373
Epoch: 170, Loss: 0.8580, Training Acc: 0.7472
Epoch: 180, Loss: 0.8398, Training Acc: 0.7521
Epoch: 190, Loss: 0.8061, Training Acc: 0.7617
Epoch: 200, Loss: 0.8188, Training Acc: 0.7562
Epoch: 210, Loss: 0.7682, Training Acc: 0.7720
Epoch: 220, Loss: 0.79

# GCN

In [ ]:
# evaluate the model on test set
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))


import time
start_time = time.time()
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.8462686538696289 seconds
Accuracy: 0.6166


# alpha = 0

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.47821044921875 seconds
Accuracy: 0.5570


# alpha = 0.1

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.4155430793762207 seconds
Accuracy: 0.5499


# alpha = 0.2

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2500901222229004 seconds
Accuracy: 0.5401


# 0.3

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.237440824508667 seconds
Accuracy: 0.5273


# 0.4

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.17484664916992188 seconds
Accuracy: 0.5222


# 0.5

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1378035545349121 seconds
Accuracy: 0.5202


# alpha = 1

In [ ]:
from collections import Counter
import random

new_x = torch.zeros(num_nodes,data.x.shape[1])
new_y = torch.zeros(num_nodes)
for i in range(len(list_of_tuples)):
    sum_node_degree = 0
    for ele in list_of_tuples[i]:
        sum_node_degree += G.out_degree(ele)
    new_node_degree = BCG.out_degree(i)
    if new_node_degree != 0:
        scalar = sum_node_degree/new_node_degree

    else:
        scalar = 1


    new_x[i,:] = scalar * torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    EC = list_of_tuples[i]
    L = []
    for ele in EC:
        L.append(int(data.y[ele].numpy()))

    keys = list(Counter(L).keys())
    frequencies = list(Counter(L).values())

    # Randomly select a key based on frequencies
    random_key = random.choices(keys, frequencies)[0]
    new_y[i] = int(random_key)

edges = list(BCG.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.09882307052612305 seconds
Accuracy: 0.5013


In [ ]:
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1014864444732666 seconds
Accuracy: 0.5004


# GAT

In [ ]:
# evaluate the model on test set
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

import time
start_time = time.time()
best_model.eval()
pred = best_model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 1.5979797840118408 seconds
Accuracy: 0.6168


# Alpha = 0

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.579277753829956 seconds
Accuracy: 0.5684


# Alpha = 0.1

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.37201881408691406 seconds
Accuracy: 0.5652


# Alpha = 0.2

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2746572494506836 seconds
Accuracy: 0.5511


# 0.3

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2083132266998291 seconds
Accuracy: 0.5353


# 0.4

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.18581390380859375 seconds
Accuracy: 0.5284


# 1

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.20810818672180176 seconds
Accuracy: 0.5190


In [ ]:
(0.5725-0.5224)/0.5725

0.08751091703056774

In [ ]:
model_save_path_GraphSage

'gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_arxiv_GraphSage.pt'

# GraphSage

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

# evaluate the model on test set
import time
start_time = time.time()
best_model.eval()
pred = best_model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 1.4965898990631104 seconds
Accuracy: 0.6111


# Alpha = 0

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

In [ ]:
# evaluate the model on test set
start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.22997593879699707 seconds
Accuracy: 0.5800


# Alpha = 0.1

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1699836254119873 seconds
Accuracy: 0.5762


# Alpha = 0.2


In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.10684919357299805 seconds
Accuracy: 0.5712


# 0.3

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.06638884544372559 seconds
Accuracy: 0.5622


# 0.4

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.08106851577758789 seconds
Accuracy: 0.5577


# 1

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.10892224311828613 seconds
Accuracy: 0.5505


In [ ]:
# evaluate the model on test set
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.164994478225708 seconds
Accuracy: 0.5691
